# Setting Environment

In [ ]:
# downloading packages

# !pip3 install numpy
# !pip3 install pandas
# !pip3 install csv
# !pip3 install openpyxl

# Main Program

## Load the Data

In [1]:
# loading the data

import pandas as pd
import numpy as np
import os
import re

# north_path = r"datasets\new_พจนานุกรมภาษาถิ่นเหนือ.xlsx"
# south_path = r"datasets\newest_พจนานุกรมภาษาถิ่นใต้.xlsx"
# isan_path = r"datasets\newest_พจนานุกรมภาษาถิ่นอีสาน.xlsx"
# orst_path = r"datasets\ราชบัณฑิตฯ"

new_orst_path = r"datasets\newest_พจนานุกรมภาษาถิ่นกลาง.xlsx"

def load_sheets(file_path, exclude_sheets):
    all_sheets = pd.ExcelFile(file_path).sheet_names
    importing_sheets = [sheet for sheet in all_sheets if sheet not in exclude_sheets]
    return pd.read_excel(file_path, sheet_name=importing_sheets, header=None)

# north_df = pd.read_excel(north_path, sheet_name=None, header=None)
# south_df = pd.read_excel(south_path, sheet_name=None, header=None)
# isan_df = pd.read_excel(isan_path, sheet_name=None, header=None)
orst_df = pd.read_excel(new_orst_path, sheet_name=None)

def load_excel(directory):
    excel_files = {}
    for filename in os.listdir(directory):
        name = os.path.splitext(filename)[0]
        excel_files[name] = pd.read_excel(os.path.join(directory, filename), header=None)
    return excel_files

# orst_df = load_excel(orst_path)

## Inspect the Data

In [2]:
# run to copy

# north = north_df.copy()
# south = south_df.copy()
# isan = isan_df.copy()
orst = orst_df.copy()

In [ ]:
# inspect the data
def inspect(num=10, *dfs):
  for df in dfs:
    for key in df.keys():
      print(f"{key}\n{df[key].head(num)}\n")

inspect(10, orst)
# inspect(10, north, south, isan, orst)

In [ ]:
# basic import

import pandas as pd
import numpy as np
import re

# df = pd.read_excel(r"", header=None)

## Cleaning

In [ ]:
# cleaning

# make dataframe two columns
def cleaning(*dfs):
    for df in dfs:
        removed = [
            'ช่อง 1', 'ลูกคำ/ ความหมาย', 'แม่คำ', 'ความหมายลูกคำ คำสื่อ',
            'headword', '2', '3']

        for key in df.keys():
            if df == orst:
                df[key] = df[key].iloc[:, :-1]
                df[key].iloc[:, 0] = df[key].apply(lambda row: row.iloc[1] if pd.isnull(row.iloc[0]) else row.iloc[0], axis=1)
                df[key] = df[key].drop(columns=[df[key].columns[1]])
                df[key] = df[key].reset_index(drop=True)
                df[key].columns = ['word', 'meaning']
                df[key] = df[key].iloc[1:]
            
            else:
                df[key] = df[key][~df[key].apply(lambda row: any(string in str(val) for string in removed for val in row), axis=1)]
                df[key] = df[key].dropna(how='all')
                df[key] = df[key].drop(df[key].columns[2:], axis=1)
                df[key] = df[key].fillna("")
                df[key].columns = ['word', 'meaning']

cleaning(orst)
# cleaning(north, south, isan, orst)

In [ ]:
# basic cleaning

def basic_cleaning(df):
    df = df.dropna(how='all')
    df = df.drop(df.columns[2:], axis=1)
    df = df.fillna("")
    df.columns = ['word', 'meaning']
    df = df.replace("", np.nan).ffill()
    return df

# basic_cleaning(df)

## Checking

In [ ]:
# checking
comma_value = []
missing_value = []
POS_value = []

def checking(df):
    regex = r',(?!(?:[^(]*\)))'
    POS = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|บุ\.|\*ส\.|สำ\.|\*สำ\.|ล\.|สำนวน\.|สำนวน\.\*|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|สรรพ\.|อ\.|อุทาน\.|\*อ\.|อุ\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|\?)(.*)$"

    # checking comma in the first column
    for key in df.keys():
        for value in df[key]['word']:
            if re.search(regex, value):
                comma_value.append(value)
        
        for value in df[key]['meaning']:
            matches = re.findall(POS, value)
            if len(matches) > 1:
                POS_value.append(value)
            elif len(matches) < 1 and value != "" and value.isspace() == False:
                missing_value.append(value)
                print(value)

    print(comma_value if len(comma_value) > 0 else None)
    print(POS_value if len(comma_value) > 0 else None)

# checking(north)
# checking(south)
# checking(isan)
checking(orst)

In [ ]:
# basic checking

comma_value = []
missing_value = []
POS_value = []

def basic_checking(df):
    regex = r',(?!(?:[^(]*\)))'
    POS = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|บุ\.|\*ส\.|สำ\.|\*สำ\.|ล\.|สำนวน\.|สำนวน\.\*|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|สรรพ\.|อ\.|อุทาน\.|\*อ\.|อุ\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|\?)(.*)$"

    # checking comma in the first column
    for value in df['word']:
        if re.search(regex, value):
            comma_value.append(value)
        
    for value in df['meaning']:
        matches = re.findall(POS, value)
        if len(matches) > 1:
            POS_value.append(value)
        elif len(matches) < 1 and value != "" and value.isspace() == False:
            missing_value.append(value)
            print(value)

    print(comma_value if len(comma_value) > 0 else None)
    print(POS_value if len(comma_value) > 0 else None)

# basic_checking(df)

##  Orst Splitting

In [ ]:
# orst splitting
# print(orst[list(orst.keys())[3]].head(), "\n")

def orst_split():
    for key in orst.keys():
        orst[key]['word'] = orst[key]['word'].str.split(',')
        orst[key] = orst[key].explode('word')
        orst[key]['word'] = orst[key]['word'].str.strip()

        # split semi-colon in the third column
        orst[key]['meaning'] = orst[key]['meaning'].str.split(';')
        orst[key] = orst[key].explode('meaning')
        orst[key]['meaning'] = orst[key]['meaning'].str.strip()

        # # filling and reseting index
        orst[key] = orst[key].ffill()
        orst[key].drop_duplicates(inplace=True)
        orst[key] = orst[key].reset_index(drop=True)

# orst_split()

# print(orst[list(orst.keys())[3]].head())

## Split POS

In [ ]:
# splitting POS

def rearrange(df):
  for key in df.keys():
    pattern = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|\*ส\.|สำ\.|\*สำ\.|สำนวน\.|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|ล\.|สรรพ\.|อ\.|\*อ\.|อุ\.|อุทาน\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|บุ\.|\?)(.*)$"
    
    df[key]["คำ"] = df[key]['word']
    df[key]["POS"] = ""
    df[key]["ความหมาย"] = ""

    if df[key]['meaning'].str.startswith((
      "น.", "*น.", "นิ.", "ก.", "*ก.", "ว.", "*ว.", "สัน.", "*สำ.", "สำ.", "ส.", "*ส.", "ความเปรียบ.", "คำถาม.", "คำยกย่อง.", "อุทาน.",
      "?", "บุพ.", "บ.", "บุ.", "บุรพ.", "สำนวน.", "สำนวน.*", "ลัก.", "สรรพ.", "อ.", "*อ.", "อุ.", "อุทาน.", "คำลงท้าย.")).any():
    
      df[key][["POS", "ความหมาย"]] = df[key]['meaning'].str.extract(pattern, expand=True)
      df[key]["ความหมาย"] = df[key].apply(lambda row: row[2] if row["POS"] == "" else row["ความหมาย"], axis=1)

    pos_mask = df[key]['meaning'].str.match(pattern, na=False)
    df[key]["POS"] = df[key]["POS"].fillna("")
    df[key].loc[~pos_mask, "ความหมาย"] = df[key].loc[~pos_mask, 'meaning']
    df[key] = df[key].iloc[:, 2:]

# rearrange(north)
# rearrange(south)
# rearrange(isan)
rearrange(orst)

def clean_whitespace(df):
  for key in df.keys():
    for column in df[key].columns:
      if df[key][column].dtype == 'object':
        df[key][column] = df[key][column].apply(
          lambda text: ' '.join(text.split()) if isinstance(text, str) else text
        )
  return df

# north = clean_whitespace(north)
# south = clean_whitespace(south)
# isan = clean_whitespace(isan)
orst = clean_whitespace(orst)

def no_whitespace(df):
  for key in df.keys():
      df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
                lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
            )

# no_whitespace(north)
# no_whitespace(south)
# no_whitespace(isan)
no_whitespace(orst)

In [ ]:
# basic splitting POS

def basic_rearrange(df):
    pattern = r"^(น\.|\*น\.|นิ\.|ก\.|\*ก\.|ว\.|\*ว\.|สัน\.|ส\.|บุรพ\.|\*ส\.|สำ\.|\*สำ\.|สำนวน\.|คำลงท้าย\.|คำยกย่อง\.|อาลปนะ\.|ลัก\.|ล\.|สรรพ\.|อ\.|\*อ\.|อุ\.|อุทาน\.|ความเปรียบ\.|คำถาม\.|บุพ\.|บ\.|บุ\.|\?)(.*)$"
    
    df = df.copy()
    
    df["คำ"] = df['word']
    df["POS"] = ""
    df["ความหมาย"] = ""

    if df['meaning'].str.startswith((
        "น.", "*น.", "นิ.", "ก.", "*ก.", "ว.", "*ว.", "สัน.", "*สำ.","สำ.", "ส.",
        "*ส.", "ความเปรียบ.", "คำถาม.", "คำยกย่อง.", "อุทาน.", "?", "บุพ.", "บ.",
        "บุ.", "บุรพ.", "สำนวน.", "สำนวน.*", "ลัก.", "สรรพ.", "อ.", "*อ.", "อุ.",
        "อุทาน.", "คำลงท้าย.")).any():
        
        df[["POS", "ความหมาย"]] = df['meaning'].str.extract(pattern, expand=True)
        df["ความหมาย"] = df.apply(lambda row: row[2] if row["POS"] == "" else row["ความหมาย"], axis=1)

    pos_mask = df['meaning'].str.match(pattern, na=False)
    df["POS"] = df["POS"].fillna("")

    df.loc[~pos_mask, "ความหมาย"] = df.loc[~pos_mask, 'meaning']
    df['ความหมาย'] = df['ความหมาย'].apply(
        lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
    )
    df = df.iloc[:, 2:]
    df = df.reset_index(drop=True)
    return df

basic_rearrange(df)

## Fill Blank Values

In [ ]:
# fill blank values
def fill_blank_words(iter=2, *dfs):
    for df in dfs:
        for _ in range(iter):
            for key in df.keys():
                df[key] = df[key].dropna(how='all')
                df[key] = df[key].infer_objects(copy=False)
                df[key] = df[key].replace(r'^\s*$', np.nan, regex=True) 
                df[key] = df[key].ffill()
                df[key].drop_duplicates(inplace=True)
                df[key] = df[key].reset_index(drop=True)
        return df

fill_blank_words(2, orst)
# fill_blank_words(2, north, south, isan, orst)

In [ ]:
def counting(df):
    def clean_text(input_text):
        pattern = r"[๐๑๒๓๔๕๖๗๘๙]|(\(.*?\))"
        cleaned_text = re.sub(pattern, "", input_text).strip()
        return cleaned_text

    for key in df.keys():
        df[key]['คำ'] = df[key]['คำ'].apply(clean_text)
                
        df[key]['occurrence'] = df[key].groupby('คำ').cumcount()
        df[key]['คำ'] = df[key]['คำ'] + " " + df[key]['occurrence'].astype(str)
        df[key] = df[key].drop(columns=['occurrence'])

        df[key]['คำ'] = df[key]['คำ'].apply(lambda x: ' '.join(x.replace("\u200b", "").split()))
                
        df[key]['ความหมาย'] = df[key]['ความหมาย'].astype(str)
        df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", "")))

        df[key] = df[key].drop_duplicates()

orst = counting(orst)

## Quick Cleaning

In [ ]:
# quick importing

import pandas as pd

path = r" "

df = pd.read_excel(path, sheet_name=None)

In [ ]:
# quick cleaning

import re

def no_more_whitespace(df):
    for key in df.keys():
        df[key].drop_duplicates(inplace=True)
        df[key] = df[key].reset_index(drop=True)
        df[key]['ความหมาย'] = df[key]['ความหมาย'].apply(
                lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", ""))
            )

# no_more_whitespace(df)

## Output

In [3]:
# save as csv

def df_to_csv(dfs, region, path):
    for sheet, df in dfs.items():
        file = f"{path}/{region}_{sheet}.csv"
        df.to_csv(file, index=False, encoding='utf-8-sig')

regions = {
    # 'north': north,
    'cen': orst,
    # 'south': south,
    # 'isan': isan,
    }

for region, dfs in regions.items():
    df_to_csv(dfs, region, r"cleaned_พจนานุกรมภาษาถิ่น\พจนานุกรมฉบับราชบัณฑิต")

In [ ]:
# save as xlsx file

def df_to_xlsx(dfs, region, path):
    with pd.ExcelWriter(f"{path}/{region}.xlsx") as writer:
        for sheet, df in dfs.items():
            df.to_excel(writer, sheet_name=sheet, index=False)

regions = {
    # 'cleaned_พจนานุกรมภาษาถิ่นเหนือ': north,
    # 'cleaned_พจนานุกรมภาษาถิ่นใต้': south,
    # 'cleaned_พจนานุกรมภาษาถิ่นอีสาน': isan,
    # 'cleaned_พจนานุกรมภาษาถิ่นกลาง': orst,
    }

for region, dfs in regions.items():
    df_to_xlsx(dfs, region, r"datasets")


## Quick Enumeration

In [4]:
# enumeration csv to csv

import pandas as pd
import os
import re
from pathlib import Path

input_path = r"cleaned_พจนานุกรมภาษาถิ่น\พจนานุกรมฉบับราชบัณฑิต"
output_path = r"cleaned_พจนานุกรมภาษาถิ่น\new_พจนานุกรมฉบับราชบัณฑิต"
Path(output_path).mkdir(parents=True, exist_ok=True)

def clean_text(input_text):
    pattern = r"[๐๑๒๓๔๕๖๗๘๙]|(\(.*?\))"
    cleaned_text = re.sub(pattern, "", input_text).strip()
    return cleaned_text

for filename in os.listdir(input_path):
    if filename.endswith('.csv'):
        input_file = os.path.join(input_path, filename)
        df = pd.read_csv(input_file)
        df['คำ'] = df['คำ'].apply(clean_text)
        
        # enumeration
        df['occurrence'] = df.groupby('คำ').cumcount()
        df['คำ'] = df['คำ'] + " " + df['occurrence'].astype(str)
        df = df.drop(columns=['occurrence'])

        # cleaning 
        df['คำ'] = df['คำ'].apply(lambda x: ' '.join(x.replace("\u200b", "").split()))
        
        df['ความหมาย'] = df['ความหมาย'].astype(str)
        df['ความหมาย'] = df['ความหมาย'].apply(lambda x: re.sub(r"^\s+|\s+$", "", x.replace("\u200b", "")))

        df = df.drop_duplicates()

        output_file = os.path.join(output_path, filename)
        df.to_csv(output_file, index=False)